In [143]:
# Install packages
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [144]:
import pandas as pd
import numpy as np
import opendatasets as od
import re
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


tf.__version__

'2.12.0'

In [145]:
# Downloads

# for nltk
nltk.download('punkt')

# Dataset
# {"username":"noahagonzo","key":"b7e4d1aedc1148c648f8fcef1ab58905"}
od.download("https://www.kaggle.com/datasets/tovarischsukhov/southparklines")

# stopwords
nltk.download('stopwords')


Skipping, found downloaded files in "./southparklines" (use force=True to force download)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [146]:
# Import training data
# TODO: Remove this to get all data ----------------------------------------------------------------------------------------------------------------
df = pd.read_csv('southparklines/All-seasons.csv', header=0, encoding='utf-8')

print(df.shape)
print(df.head())

(70896, 4)
  Season Episode Character                                               Line
0     10       1      Stan         You guys, you guys! Chef is going away. \n
1     10       1      Kyle                        Going away? For how long?\n
2     10       1      Stan                                         Forever.\n
3     10       1      Chef                                  I'm sorry boys.\n
4     10       1      Stan  Chef said he's been bored, so he joining a gro...


In [147]:
# Define a function for preprocessing text to remove special characters and alter format of expressive words
def preprocess_text(text):
  text = text.lower()

  # Modify text
  text = text.replace("\n","")
  punc = [".", "!", "?", ","] # Remove punctuation that messes up regex
  for p in punc:
    text = re.sub("[" + p + "]", " " + p, text)
  text = re.sub(r"o[h]{2,}", "oh", text)
  text = re.sub(r"a[h]{2,}", "oh", text)

  # Tokenize to remove stopwords and punctuation
  tokens = word_tokenize(text)
  stop_words = set(stopwords.words('english')) # TODO: try removing stopwords and keeping punctionation
  tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation] 

  return text


In [148]:
# Preprocess text
text = []

for line in df.Line:
  text.append(preprocess_text(line))


In [149]:
# Create a dictionary for frequencies
vocab = {}
for line in text:
  for word in line.split():
    if word not in vocab:
      vocab[word] = 1
    else:
      vocab[word] += 1

print("Size of vocab:", len(vocab))
print(vocab)

Size of vocab: 33607
{'you': 25056, 'guys': 1963, ',': 62433, '!': 42578, 'chef': 412, 'is': 9765, 'going': 1606, 'away': 641, '.': 81967, '?': 23761, 'for': 5586, 'how': 2297, 'long': 490, 'forever': 109, "i'm": 4874, 'sorry': 837, 'boys': 892, 'said': 742, "he's": 1316, 'been': 1186, 'bored': 17, 'so': 3546, 'he': 2711, 'joining': 32, 'a': 15164, 'group': 98, 'called': 302, 'the': 24938, 'super': 126, 'adventure': 46, 'club': 114, 'wow': 512, 'what': 7053, 'kind': 436, 'of': 9357, 'questions': 92, 'do': 5017, 'think': 2216, 'adventuring': 1, 'around': 558, 'world': 589, 'gonna': 2553, 'answer': 130, "what's": 1022, 'meaning': 28, 'life': 471, 'why': 1597, 'are': 5360, 'we': 9177, 'here': 3355, 'i': 17448, 'hope': 239, "you're": 2680, 'making': 268, 'right': 3419, 'choice': 116, 'miss': 146, 'him': 1738, 'and': 13222, '.and': 112, "don't": 4482, 'know': 3455, 'to': 19723, 'tell': 1043, 'dude': 2007, 'go': 3141, 'on': 6013, 'was': 3091, 'our': 2595, 'fuh': 3, '.f-ffriend': 1, 'will': 1

In [150]:
# MAYBE: SEt a threshold so only words that are used more than 3 times are included in the vocab


In [151]:
# Create tokenizer
tokenizer = Tokenizer(num_words=len(vocab), oov_token="<OOV>") #num_word = max number of words to keep, based on word frequency
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index

In [153]:
# Create sequences
maxlen=300
sequences = tokenizer.texts_to_sequences(text)
padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding="post")

In [154]:
# Create training data
x_train = padded_sequences
y_train = padded_sequences

In [155]:
# Build model
model = models.Sequential()
model.add(layers.Input(shape=(maxlen,)))
model.add(layers.Dense(maxlen, input_dim=len(vocab), kernel_initializer='normal', activation='relu'))
model.add(layers.Dense(maxlen, input_dim=len(vocab), kernel_initializer='normal', activation='relu'))

In [156]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [157]:
# Train model
history = model.fit(x_train, y_train,batch_size=100,epochs=10,verbose=1, validation_split=0.1)

Epoch 1/10
639/639 [==============================] - 8s 12ms/step - loss: -258.0129 - accuracy: 0.1418 - val_loss: -296.5354 - val_accuracy: 0.0189
Epoch 2/10
639/639 [==============================] - 6s 9ms/step - loss: -356.1420 - accuracy: 0.0231 - val_loss: -382.9811 - val_accuracy: 0.0241
Epoch 3/10
639/639 [==============================] - 7s 11ms/step - loss: -394.7861 - accuracy: 0.0293 - val_loss: -387.6612 - val_accuracy: 0.0126
Epoch 4/10
639/639 [==============================] - 5s 9ms/step - loss: -398.2029 - accuracy: 0.0141 - val_loss: -394.8094 - val_accuracy: 0.0169
Epoch 5/10
639/639 [==============================] - 7s 11ms/step - loss: -403.7710 - accuracy: 0.0245 - val_loss: -398.4701 - val_accuracy: 0.0330
Epoch 6/10
639/639 [==============================] - 7s 11ms/step - loss: -405.7530 - accuracy: 0.0322 - val_loss: -398.5429 - val_accuracy: 0.0300
Epoch 7/10
639/639 [==============================] - 7s 11ms/step - loss: -404.1624 - accuracy: 0.0291 - va

In [159]:
# Define function to predict answer
def predict_answer(model, tokenizer, question):
    # Preprocess question
    question = preprocess_text(question)
    # Convert question to sequence
    sequence = tokenizer.texts_to_sequences([question])
    # Pad sequence
    padded_sequence = pad_sequences(sequence, maxlen=maxlen, padding="post", truncating="post")
    # Predict answer
    pred = model.predict(padded_sequence)[0]
    # Get index of highest probability
    idx = np.argmax(pred)
    # Get answer
    answer = tokenizer.index_word[idx]
    return answer

# Start chatbot
while True:
    question = input('You: ')
    answer = predict_answer(model, tokenizer, question)
    print('Chatbot:', answer)

You: Hi
1/1 [==============================] - 0s 23ms/step
Chatbot: it
You: no
1/1 [==============================] - 0s 26ms/step
Chatbot: it
You: Cartman
1/1 [==============================] - 0s 25ms/step
Chatbot: it
You: Fat
1/1 [==============================] - 0s 27ms/step
Chatbot: it
You: mecha
1/1 [==============================] - 0s 24ms/step
Chatbot: it


KeyboardInterrupt: ignored